<a href="https://colab.research.google.com/github/Davidxswang/ML/blob/master/Note_1_TensorFlow_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the packages and check the environment
[Refer to the link for full tutorial](https://www.tensorflow.org/datasets/overview)

**Be careful, in the tutorial, two lines of code cannot work:**
- **fig = tfds.show_examples(ds, info)**
- **print(info.splits['train'].filenames)**


In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [3]:
! pwd
# mnist dataset is stored in the path below
#! mkdir /content/drive/My\ Drive/colab/mnist

/content


# How to use tfds

## See what datasets are included in the tfds



In [0]:
# To see what datasets are available (included in tfds)
if False:
  tfds.list_builders()

## How to load the dataset, what type of structure the data has

There are three key things you need to know:
1. as_supervised argument of load method
2. batch_size argument of load method
3. as_numpy() method of tdfs

The 1 and 2 will affect the output structure of the load method.

The 3 can convert the tf.data.Dataset into Generator[np.ndarray] and tf.Tensor into np.ndarray.

In [65]:
# all the configs
as_supervised = True
# if True, elements in ds_train are tuple, if False, elements are dict

batch_size = None
# -1, load the full batch into a tuple/dict, ds_train will be a tuple if used with as_supervised=True, a dict if as_supervised=False
# None, ds_train will be tf.data.Dataset, no matter what as_supervised is


# load the data
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train[:80%]', 'train[80%:]', 'test'],
    shuffle_files=True,
    as_supervised=as_supervised,     
    with_info=True,
    data_dir='/content/drive/My Drive/colab/mnist',
    batch_size=batch_size
)
print(ds_info)

print('We can access the features of the dataset by this way:')
print(ds_info.features.shape, ds_info.features.dtype)
print(ds_info.features['image'].shape, ds_info.features['image'].dtype)
print(ds_info.features['label'].num_classes, ds_info.features['label'].names, ds_info.features['label'].int2str(7), ds_info.features['label'].str2int('7'))
print("\n")

print('We can access the split of the dataset by this way:')
print(ds_info.splits)
print(list(ds_info.splits.keys()))
print(dir(ds_info.splits['train']))
print(ds_info.splits['train'].num_examples)
print(ds_info.splits['train'].file_instructions)
print(ds_info.splits['train'].file_instructions[0]['filename'])
print(ds_info.splits['train'].num_shards)
print(ds_info.splits['train[15%:25%]'].num_examples)
print(ds_info.splits['train[15%:25%]'].file_instructions)
print("\n")




print('ds_train type is (tuple, dict or tf.data.Dataset): ', type(ds_train))
# take a look at the internal structure of the data
if batch_size == -1:
  # ds_train will be dict or tuple
  if as_supervised:
    print('ds_train will be a tuple (images, labels)')
    print('How many images in ds_train: ', len(ds_train[0]))
    print('Type of elements in ds_train\'s images: ', type(ds_train[0][0]))
    print('Shape of the image: ', ds_train[0][0].shape)
    print('Type of elements in ds_train\'s labels: ', type(ds_train[1][0]))
    print('The value of the label: ', ds_train[1][0].numpy())
    print('\n')
    print('We can use tfds.as_numpy(something) to convert something into: Generator[np.ndarray] from tf.data.Dataset, or np.ndarray from tf.Tensor')
    print('We can convert ds_train[0] which is originally a tf.Tensor, now the type is: ', type(tfds.as_numpy(ds_train[0])))
    print('We can also convert the ds_train (a tuple) directly, after convert it\'s still a', type(tfds.as_numpy(ds_train)), ', the ds_train[0] is now the type of: ', type(tfds.as_numpy(ds_train)[0]))
  else:
    print('ds_train will be a dict {\'image\': image, \'label\': label}')
    print('Type of ds_train[\'image\']: ', type(ds_train['image']))
    print('Shape of ds_train["image"]: ', ds_train['image'].shape)
    print('Type of ds_train[\'label\']: ', type(ds_train['label']))
    print('Shape of ds_train["label"]: ', ds_train['label'].shape)
    print('\n')
    print('We can convert ds_train["image"] which is originally a tf.Tensor, now the type is: ', type(tfds.as_numpy(ds_train["image"])))
    print('We can also convert the ds_train (a dict) directly, after convert it\'s still a', type(tfds.as_numpy(ds_train)), ', the ds_train["image"] is now the type of: ', type(tfds.as_numpy(ds_train)["image"]))
elif batch_size is None:
  print('ds_train is a tf.data.Dataset object')
  ds = ds_train.take(1)
  for data in ds:
    if as_supervised:
      print('The element in ds_train is tuple, as you can see: ', type(data))
      print('The type of the first element of this tuple: ', type(data[0]))
      print('The shape of this image (tf.Tensor): ', data[0].shape)
      print('The type of the second element of this tuple: ', type(data[1]))
      print('The shape of this label (tf.Tensor): ', data[1].shape)
      print('The value of this label (tf.Tensor): ', data[1].numpy())
      print('\n')
      print('We can conver the tf.data.Dataset using tfds.as_numpy() method, the type will be: ', type(tfds.as_numpy(ds)))
      print('The element from this generator will be the type of tuple: ', type(list(tfds.as_numpy(ds))[0]))
      print('The first element (an image) of this tuple will be the type of np.ndarray: ', type(list(tfds.as_numpy(ds))[0][0]))
      print('The shape of this image will be: ', list(tfds.as_numpy(ds))[0][0].shape)
    else:
      print('The element in ds_train is dict, as you can see: ', type(data))
      print('The keys are: ', list(data.keys()))
      print('The type of the first element of this dict: ', type(data['image']))
      print('The shape of this image (tf.Tensor): ', data['image'].shape)
      print('The type of the second element of this dict: ', type(data['label']))
      print('The shape of this label (tf.Tensor): ', data['label'].shape)
      print('The value of this label (tf.Tensor): ', data['label'].numpy())
      print('\n')
      print('We can conver the tf.data.Dataset using tfds.as_numpy() method, the type will be: ', type(tfds.as_numpy(ds)))
      print('The element from this generator will be the type of dict: ', type(list(tfds.as_numpy(ds))[0]))
      print('The image of this dict will be the type of np.ndarray: ', type(list(tfds.as_numpy(ds))[0]['image']))
      print('The shape of this image will be: ', list(tfds.as_numpy(ds))[0]['image'].shape)

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

We can access the features of the dataset by this way:
{'image': (28, 28, 1), 'label': ()} {'image': tf.uint8, 'label': tf.int64}
(28, 28, 1) <dtype: 'uint8'>
10 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 7 7


## How to use the dataset to train a neural network
[Refer to the full tutorial](https://www.tensorflow.org/datasets/keras_example)
